In [20]:
%load_ext autoreload
%autoreload 2
from os.path import join
from tqdm import tqdm
import pandas as pd
import numpy as np
import clin.llm
import clin.eval.med_status
import clin.parse
import openai
openai.api_key_path = '/home/chansingh/.OPENAI_KEY'
from typing import List
results_dir = '../results-mega/'
# results_dir = '../results_tmp/'
from clin.config import PATH_REPO
import datasets
import imodelsx.viz
import imodelsx.process_results
def viz_blue_rows(df):
    return (
        df
        # highlight rows 2 and 4 including the index
        .style.apply(lambda x: ['background: lightblue' if 'self-verified' in x.name else '' for i in x], axis=1)
        # bold index label for row 2
        .apply(lambda x: ['font-weight: bold' if 'self-verified' in x.name else '' for i in x], axis=1)
        .format(precision=3)
    )   
r = imodelsx.process_results.get_results_df(results_dir, use_cached=False)
r = r[[col for col in r.columns if not col in ['checkpoint_verify', 'role_verify']]]
r = r[r.use_megaprompt == 1]
# r = imodelsx.process_results.average_over_seeds(r, experiment_filename=join(PATH_REPO, 'experiments', 'eval_model.py'))

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


100%|██████████| 6/6 [00:00<00:00, 1564.55it/s]


In [21]:
r

,dataset_name,seed,save_dir,checkpoint,use_megaprompt,n_shots,use_cache,save_dir_unique,extracted_strs,recall___original,precision___original,f1___original,dict_original,list_original
0,medication_status,1,/home/chansingh/clinical-extract-and-check/res...,gpt-3.5-turbo,1,5,1,/home/chansingh/clinical-extract-and-check/res...,"[""dobutamine"" (active)\n- ""Nipride"" (active)\n...",0.920588,0.650728,0.762485,"[{'dobutamine': 'active', 'Nipride': 'active',...",NaN
1,medication_status,1,/home/chansingh/clinical-extract-and-check/res...,gpt-4-0314,1,5,1,/home/chansingh/clinical-extract-and-check/res...,"[""dobutamine"" (active)\n- ""Nipride"" (active)\n...",0.888235,0.782383,0.831956,"[{'dobutamine': 'active', 'Nipride': 'active',...",NaN
2,ebm,1,/home/chansingh/clinical-extract-and-check/res...,gpt-3.5-turbo,1,5,1,/home/chansingh/clinical-extract-and-check/res...,NaN,0.378505,0.292419,0.329939,NaN,"[[budesonide Turbuhaler, budesonide aqua], [gr..."
3,ebm,1,/home/chansingh/clinical-extract-and-check/res...,text-davinci-003,1,5,1,/home/chansingh/clinical-extract-and-check/res...,NaN,0.481308,0.422131,0.449782,NaN,"[[budesonide Turbuhaler, budesonide aqua], [gr..."
4,ebm,1,/home/chansingh/clinical-extract-and-check/res...,gpt-4-0314,1,5,1,/home/chansingh/clinical-extract-and-check/res...,NaN,0.434579,0.395745,0.414254,NaN,"[[budesonide Turbuhaler, budesonide aqua], [gr..."
5,medication_status,1,/home/chansingh/clinical-extract-and-check/res...,text-davinci-003,1,5,1,/home/chansingh/clinical-extract-and-check/res...,"[ ""dobutamine"" (active)\n- ""Nipride"" (active)\...",0.926471,0.907781,0.917031,"[{'dobutamine': 'active', 'Nipride': 'active',...",NaN


In [22]:
r_med = r[r.dataset_name == 'medication_status']
# dset = datasets.load_dataset('mitclinicalml/clinical-ie', 'medication_status')
# df_val = pd.DataFrame.from_dict(dset['validation'])
# df = pd.DataFrame.from_dict(dset['test'])
# # df = pd.concat([val, test])
# nums = np.arange(len(df)).tolist()
# np.random.default_rng(seed=13).shuffle(nums)
# # dfe = df.iloc[nums]
# r_med = clin.eval.med_status.add_status_eval(r_med, df)

cols = {
    'f1___original': 'Medication name, original',
    # 'f1___ov_pv_ev': 'Medication name, self-verified',
    # 'status_f1_macro_cond___original': 'Medication status, original',
    # 'status_f1_macro_cond___sv': 'Medication status, self-verified',
}
rt_med_status = r_med.groupby(['checkpoint'])[list(cols.keys())].mean().rename(columns=cols).round(3).T
rt_med_status_sem = r_med.groupby(['checkpoint'])[list(cols.keys())].sem(ddof=0).rename(columns=cols).round(3).T
viz_blue_rows(rt_med_status)

checkpoint,gpt-3.5-turbo,gpt-4-0314,text-davinci-003
"Medication name, original",0.762,0.832,0.917


In [23]:
rt_med_status

checkpoint,gpt-3.5-turbo,gpt-4-0314,text-davinci-003
"Medication name, original",0.762,0.832,0.917


In [24]:
r_ebm = r[r.dataset_name == 'ebm']
cols = {
    'f1___original': 'Clinical trial arm, original',
    # 'f1___ov_pv_ev': 'Clinical trial arm, self-verified',
}
rt_ebm = r_ebm.groupby(['checkpoint'])[list(cols.keys())].mean().rename(columns=cols).round(3).T
rt_ebm_sem = r_ebm.groupby(['checkpoint'])[list(cols.keys())].sem(ddof=0).rename(columns=cols).round(3).T
viz_blue_rows(rt_ebm)

checkpoint,gpt-3.5-turbo,gpt-4-0314,text-davinci-003
"Clinical trial arm, original",0.330,0.414,0.450


In [25]:
for k in rt_ebm.columns:
    rt_ebm[k] = rt_ebm[k].astype(str) + "\err{" + rt_ebm_sem[k].astype(str) + "}"
for k in rt_med_status.columns:
    rt_med_status[k] = (
        rt_med_status[k].astype(str) + "\err{" + rt_med_status_sem[k].astype(str) + "}"
    )
rt = pd.concat((rt_ebm, rt_med_status))
rt.columns = rt.columns.map(lambda x: imodelsx.viz.CHECKPOINTS_RENAME_DICT.get(x, x))
viz_blue_rows(rt)

<>:2: DeprecationWarning: invalid escape sequence '\e'
<>:5: DeprecationWarning: invalid escape sequence '\e'
<>:2: DeprecationWarning: invalid escape sequence '\e'
<>:5: DeprecationWarning: invalid escape sequence '\e'
/tmp/ipykernel_3649349/2639324598.py:2: DeprecationWarning: invalid escape sequence '\e'
  rt_ebm[k] = rt_ebm[k].astype(str) + "\err{" + rt_ebm_sem[k].astype(str) + "}"
/tmp/ipykernel_3649349/2639324598.py:5: DeprecationWarning: invalid escape sequence '\e'
  rt_med_status[k].astype(str) + "\err{" + rt_med_status_sem[k].astype(str) + "}"


checkpoint,ChatGPT,GPT-4,GPT-3
"Clinical trial arm, original",0.33\err{0.0},0.414\err{0.0},0.45\err{0.0}
"Medication name, original",0.762\err{0.0},0.832\err{0.0},0.917\err{0.0}


In [19]:
print(rt.style.format(precision=3).to_latex(hrules=True))

\begin{tabular}{llll}
\toprule
checkpoint & GPT-4 & GPT-3 & ChatGPT \\
\midrule
Clinical trial arm, original & 0.437\err{0.0} & 0.536\err{0.0} & nan \\
Medication name, original & 0.911\err{0.0} & 0.938\err{0.0} & 0.838\err{0.0} \\
\bottomrule
\end{tabular}

